In [18]:
# Importación de librerías
import pandas as pd  # Para manipulación y análisis de datos
from sklearn.model_selection import train_test_split  # Para dividir el conjunto de datos en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Para estandarizar características
from tensorflow.keras.models import Sequential  # Para inicializar modelos secuenciales
from tensorflow.keras.layers import Dense, Input  # Para definir capas densamente conectadas en la red neuronal
from sklearn.impute import SimpleImputer  # Para imputar valores faltantes
import numpy as np  # Para cálculos numéricos
from sklearn.metrics import mean_squared_error, r2_score  # Para calcular métricas de rendimiento del modelo

In [19]:
# Cargar datos
df = pd.read_csv("jugadores_fantasy_migate.csv")


# Imprimir las columnas existentes y la columna posicion 
print(df.columns)
print("\n")
df

Index(['Nombre', 'Precio', 'Media', 'Total_puntos', 'posicion', 'equipo',
       'Partidos', 'Minutos', 'Goles', 'Asistencias', 'Asistencias sin Gol',
       'Centros', 'Despejes', 'Remates', 'Tiros', 'Recuperaciones', 'Perdidas',
       'Penaltis fallados', 'Goles en contra', 'Rojas', 'Paradas',
       'Penaltis cometidos', 'Amarillas', 'Seguna Amarilla',
       'Penaltis provocados', 'Penaltis parados', 'Goles en PP'],
      dtype='object')




,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Penaltis fallados,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,13.0,1025.0,11.0,2.0,...,0.0,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,T. Kubo,20904000,8.2,147,mediocentro,Real Sociedad de Fútbol,14.0,970.0,5.0,2.0,...,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,13.0,1071.0,9.0,1.0,...,0.0,11.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,I. Williams,18358000,7.7,139,delantero,Athletic Club,15.0,1233.0,6.0,3.0,...,0.0,16.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,B. Mayoral,16992000,7.3,138,delantero,Getafe Club de Fútbol,14.0,1170.0,0.0,0.0,...,0.0,17.0,0.0,46.0,0.0,2.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,P. Reina,160000,0.0,0,portero,Villarreal Club de Fútbol,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,T. Fuidias,160000,0.0,0,portero,Girona Fútbol Club,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,T. Villa,160000,0.0,0,mediocentro,Girona Fútbol Club,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,U. Ropero,160000,0.0,0,mediocentro,Deportivo Alavés,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Dividir los datos en características (X) y el objetivo (y)
X_Todo = df[['Media', 'Partidos', 'Minutos', 'Goles', 'Asistencias', 'Asistencias sin Gol', 'Centros', 'Despejes', 'Remates', 'Tiros', 'Recuperaciones', 'Perdidas', 'Penaltis fallados', 'Goles en contra', 'Rojas', 'Paradas', 'Penaltis cometidos', 'Amarillas', 'Seguna Amarilla', 'Penaltis provocados', 'Penaltis parados', 'Goles en PP']]
y_puntos = df['Total_puntos']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_puntos, X_test_puntos, y_puntos_train, y_puntos_test = train_test_split(X_Todo, y_puntos, test_size=0.2, random_state=42)


# Escalar las características
scaler = StandardScaler()
X_train_puntos_scaled = scaler.fit_transform(X_train_puntos)
X_test_puntos_scaled = scaler.transform(X_test_puntos)

In [21]:
# Crear el modelo para predecir puntos
model_puntos = Sequential([
    Input(shape=(X_train_puntos_scaled.shape[1],)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Configuración del modelo para que utilice el algoritmo de optimización 
model_puntos.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo para predecir puntos
model_puntos.fit(X_train_puntos_scaled, y_puntos_train, epochs=50, batch_size=32, verbose=0)


In [22]:
# Hacer predicciones para puntos y precio
puntos_predictions = model_puntos.predict(X_test_puntos_scaled).flatten()

# Crear un DataFrame con las predicciones y el índice original
predictions_df = pd.DataFrame({
    'Predicted_Points': puntos_predictions,
}, index=X_test_puntos.index)

# Unir las predicciones con el DataFrame original usando el índice
df_with_predictions = df.join(predictions_df, how='inner')

# Ordenar el DataFrame de forma descendente por precio predicho y puntos predichos
df_sorted = df_with_predictions.sort_values(by=['Predicted_Points'], ascending=False)

# Impresión de la lista de jugadores ordenados por mayor precio y puntos predichos
print("Lista de jugadores ordenados por mayor precio y puntos predichos:")
print(df_sorted[['Nombre', 'Predicted_Points']])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Lista de jugadores ordenados por mayor precio y puntos predichos:
            Nombre  Predicted_Points
0    J. Bellingham        186.793243
2     A. Griezmann        132.097397
9       A. Budimir        107.031807
30       B. Méndez         84.999245
72       V. Muriqi         83.250877
..             ...               ...
481     P. Cuéllar          7.189806
474    T. Courtois          7.189806
471      I. Moriba          7.189806
470     J. Cardoso          7.189806
501       P. Reina          7.189806

[102 rows x 2 columns]


In [23]:
# Calcular el Error Cuadrático Medio (MSE) para puntos y precio
mse_puntos = mean_squared_error(y_puntos_test, puntos_predictions)

# Calcular el coeficiente de determinación (R^2) para puntos y precio
r2_puntos = r2_score(y_puntos_test, puntos_predictions)

print("Calculo del error cuadratico y coeficiente de determinacion por puntos:")
print(f"MSE para puntos: {mse_puntos:.2f}")
print(f"R^2 para puntos: {r2_puntos:.2f}")

Calculo del error cuadratico y coeficiente de determinacion por puntos:
MSE para puntos: 339.37
R^2 para puntos: 0.76


In [24]:
# Crear DataFrames para los top 10 jugadores por precio y puntos predichos
top_10_puntos = df_sorted.nlargest(10, 'Predicted_Points')

# Filtrar jugadores con la posición de delantero
delanteros = df_with_predictions[df_with_predictions['posicion'] == 'delantero']
mediocentro = df_with_predictions[df_with_predictions['posicion'] == 'mediocentro']
portero = df_with_predictions[df_with_predictions['posicion'] == 'portero']
defensa = df_with_predictions[df_with_predictions['posicion'] == 'defensa']

# Crear DataFrames para los top 10 Delanteros por precio y puntos predichos
top_10_puntos_Delanteros = delanteros.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Mediocentros por precio y puntos predichos
top_10_puntos_Mediocentro = mediocentro.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Porteros por precio y puntos predichos
top_10_puntos_Portero = portero.nlargest(10, 'Predicted_Points')

# Crear DataFrames para los top 10 Defensas por precio y puntos predichos
top_10_puntos_Defensa = defensa.nlargest(10, 'Predicted_Points')


In [25]:
# Guardar los DataFrames en archivos CSV
top_10_puntos.to_csv('general_predicción.csv', index=False)

top_10_puntos_Delanteros.to_csv('delantero_predicción.csv', index=False)

top_10_puntos_Mediocentro.to_csv('mediocentro_predicción.csv', index=False)

top_10_puntos_Portero.to_csv('portero_predicción.csv', index=False)

top_10_puntos_Defensa.to_csv('defensa_predicción.csv', index=False)

In [26]:
# Mostrar los DataFrames creados
print("Top 10 jugadores por puntos predichos:")
print(top_10_puntos[['Nombre', 'Predicted_Points']])


print("Top 10 delanteros por puntos predichos:")
print(top_10_puntos_Delanteros[['Nombre', 'Predicted_Points']])


print("Top 10 mediocentros por puntos predichos:")
print(top_10_puntos_Mediocentro[['Nombre', 'Predicted_Points']])


print("Top 10 porteros por puntos predichos:")
print(top_10_puntos_Portero[['Nombre', 'Predicted_Points']])


print("Top 10 defensas por puntos predichos:")
print(top_10_puntos_Defensa[['Nombre',  'Predicted_Points']])

Top 10 jugadores por puntos predichos:
            Nombre  Predicted_Points
0    J. Bellingham        186.793243
2     A. Griezmann        132.097397
9       A. Budimir        107.031807
30       B. Méndez         84.999245
72       V. Muriqi         83.250877
77      R. De Paul         81.006096
137   Ó. De Marcos         80.256409
55      Y. Herrera         79.518852
76      J. Cancelo         79.261124
208    M. Diakhaby         75.181854
Top 10 delanteros por puntos predichos:
              Nombre  Predicted_Points
2       A. Griezmann        132.097397
9         A. Budimir        107.031807
72         V. Muriqi         83.250877
22         G. Moreno         69.315399
73         F. Torres         59.428417
11             Sávio         58.563282
84       L. Ramazani         58.227371
90          J. Félix         56.822006
173  J. Luis Morales         54.945099
75          M. Uzuni         54.048374
Top 10 mediocentros por puntos predichos:
            Nombre  Predicted_Points
0    J

In [27]:
top_10_puntos

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,13.0,1025.0,11.0,2.0,...,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,186.793243
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,13.0,1071.0,9.0,1.0,...,11.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,132.097397
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,14.0,966.0,1.0,3.0,...,10.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,107.031807
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,10.0,866.0,2.0,1.0,...,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,84.999245
72,V. Muriqi,12454000,6.8,81,delantero,Real Club Deportivo Mallorca,12.0,942.0,1.0,0.0,...,13.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,83.250877
77,R. De Paul,11692000,5.0,80,mediocentro,Club Atlético de Madrid,14.0,1156.0,1.0,1.0,...,20.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,81.006096
137,Ó. De Marcos,6181000,4.6,65,defensa,Athletic Club,14.0,1125.0,0.0,0.0,...,18.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,80.256409
55,Y. Herrera,11017000,5.7,86,mediocentro,Girona Fútbol Club,11.0,729.0,1.0,5.0,...,11.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,79.518852
76,J. Cancelo,16261000,5.0,80,defensa,Fútbol Club Barcelona,14.0,1258.0,0.0,1.0,...,18.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,79.261124
208,M. Diakhaby,1456000,4.2,50,defensa,Valencia Club de Fútbol,14.0,977.0,0.0,0.0,...,19.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,75.181854


In [28]:
top_10_puntos_Delanteros

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points
2,A. Griezmann,24384000,7.6,144,delantero,Club Atlético de Madrid,13.0,1071.0,9.0,1.0,...,11.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,132.097397
9,A. Budimir,15660000,6.7,127,delantero,Club Atlético Osasuna,14.0,966.0,1.0,3.0,...,10.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,107.031807
72,V. Muriqi,12454000,6.8,81,delantero,Real Club Deportivo Mallorca,12.0,942.0,1.0,0.0,...,13.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,83.250877
22,G. Moreno,18341000,6.5,104,delantero,Villarreal Club de Fútbol,11.0,804.0,0.0,0.0,...,20.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,69.315399
73,F. Torres,8080000,4.5,81,delantero,Fútbol Club Barcelona,12.0,948.0,1.0,0.0,...,14.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,59.428417
11,Sávio,18030000,6.5,123,delantero,Girona Fútbol Club,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.563282
84,L. Ramazani,3171000,4.2,79,delantero,Unión Deportiva Almería,13.0,751.0,0.0,0.0,...,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,58.227371
90,J. Félix,14964000,4.8,76,delantero,Fútbol Club Barcelona,12.0,766.0,0.0,3.0,...,14.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,56.822006
173,J. Luis Morales,4457000,4.0,56,delantero,Villarreal Club de Fútbol,14.0,384.0,3.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.945099
75,M. Uzuni,3324000,4.3,81,delantero,Granada Club de Fútbol,13.0,653.0,1.0,3.0,...,16.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,54.048374


In [29]:
top_10_puntos_Mediocentro

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points
0,J. Bellingham,24262000,10.2,173,mediocentro,Real Madrid Club de Fútbol,13.0,1025.0,11.0,2.0,...,8.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,186.793243
30,B. Méndez,15078000,5.7,97,mediocentro,Real Sociedad de Fútbol,10.0,866.0,2.0,1.0,...,9.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,84.999245
77,R. De Paul,11692000,5.0,80,mediocentro,Club Atlético de Madrid,14.0,1156.0,1.0,1.0,...,20.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,81.006096
55,Y. Herrera,11017000,5.7,86,mediocentro,Girona Fútbol Club,11.0,729.0,1.0,5.0,...,11.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,79.518852
18,F. Valverde,20115000,5.7,108,mediocentro,Real Madrid Club de Fútbol,14.0,1118.0,1.0,2.0,...,6.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,74.627243
78,J. Guerra,8999000,4.7,80,mediocentro,Valencia Club de Fútbol,14.0,1078.0,2.0,1.0,...,13.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,73.938507
68,V. Tsygankov,14504000,5.9,82,mediocentro,Girona Fútbol Club,12.0,887.0,1.0,5.0,...,20.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,72.389648
104,D. Parejo,11226000,4.1,74,mediocentro,Villarreal Club de Fútbol,13.0,730.0,1.0,2.0,...,8.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,67.708130
131,G. Gumbau,4257000,3.9,67,mediocentro,Granada Club de Fútbol,12.0,993.0,0.0,0.0,...,13.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,66.924728
203,L. Torró,1951000,3.6,51,mediocentro,Club Atlético Osasuna,12.0,980.0,0.0,0.0,...,16.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,65.764969


In [30]:
top_10_puntos_Portero

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points
491,A. López,164000,0.0,0,portero,Granada Club de Fútbol,14.0,813.0,3.0,1.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.818287
436,D. Mariño,189000,3.0,3,portero,Unión Deportiva Almería,14.0,90.0,0.0,0.0,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,34.971058
392,R. Fernández,160000,3.3,13,portero,Granada Club de Fútbol,13.0,373.0,2.0,1.0,...,3.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,33.439602
474,T. Courtois,368000,0.0,0,portero,Real Madrid Club de Fútbol,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.189806
481,P. Cuéllar,187000,0.0,0,portero,Real Club Deportivo Mallorca,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.189806
496,I. Grbić,160000,0.0,0,portero,Club Atlético de Madrid,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.189806
497,J. Doménech,160000,0.0,0,portero,Valencia Club de Fútbol,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.189806
501,P. Reina,160000,0.0,0,portero,Villarreal Club de Fútbol,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.189806


In [31]:
top_10_puntos_Defensa

,Nombre,Precio,Media,Total_puntos,posicion,equipo,Partidos,Minutos,Goles,Asistencias,...,Goles en contra,Rojas,Paradas,Penaltis cometidos,Amarillas,Seguna Amarilla,Penaltis provocados,Penaltis parados,Goles en PP,Predicted_Points
137,Ó. De Marcos,6181000,4.6,65,defensa,Athletic Club,14.0,1125.0,0.0,0.0,...,18.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,80.256409
76,J. Cancelo,16261000,5.0,80,defensa,Fútbol Club Barcelona,14.0,1258.0,0.0,1.0,...,18.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,79.261124
208,M. Diakhaby,1456000,4.2,50,defensa,Valencia Club de Fútbol,14.0,977.0,0.0,0.0,...,19.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,75.181854
63,A. Witsel,10736000,4.7,85,defensa,Club Atlético de Madrid,12.0,1010.0,1.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.198380
79,S. Cardona,11393000,4.4,80,defensa,Unión Deportiva Las Palmas,14.0,705.0,1.0,0.0,...,12.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,64.448502
124,D. Rico,10002000,4.9,68,defensa,Getafe Club de Fútbol,11.0,934.0,1.0,3.0,...,15.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,61.504166
281,O. Alderete,597000,2.8,37,defensa,Getafe Club de Fútbol,13.0,568.0,0.0,1.0,...,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,45.436432
218,R. Marín,2507000,3.3,49,defensa,Deportivo Alavés,13.0,546.0,0.0,0.0,...,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,44.497238
93,M. Mármol,10103000,4.5,76,defensa,Unión Deportiva Las Palmas,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.685173
204,C. Azpilicueta,1627000,3.2,51,defensa,Club Atlético de Madrid,14.0,470.0,1.0,1.0,...,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,40.573189


In [32]:
########################################################################################################################